# imports

In [1]:
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd
from matplotlib.lines import Line2D

# load data and create pandas table

In [ ]:
ls *_results

In [ ]:
res_standard = pd.read_pickle("new_targets_results")
res_deeprelu = pd.read_pickle("deeprelu_results")
# res_middlelinear = pd.read_pickle("middlelinear_SIM_results")
print(res_deeprelu.shape,res_standard.shape)#,res_middlelinear.shape)

In [ ]:
res_standard[(res_standard.r==1) * (res_standard.sigma<=0.25)].shape

In [5]:
res = pd.concat([
    res_standard[(res_standard.r==1) * (res_standard.sigma<=0.25)], #filter to just the r and sigma vals we ran everything on
    res_deeprelu[res_deeprelu.L>2], #only look at the actually deep models with these architectures
    # res_middlelinear[res_middlelinear.L>2]
])
res.loc[res.L==2,"Activations"] = "Shallow"

In [ ]:
res.Activations.unique()

In [7]:
res.loc[res.Activations == "standard","Activations"] = "Linear Layers then ReLU"
res.loc[res.Activations == "relus","Activations"] = "Deep ReLU"
# res.loc[res.Activations == "middlelinear","Activations"] = "ReLU then Linear Layers then ReLU"

In [ ]:
res.Activations.unique()

In [ ]:
res.shape

# Check if Final Training Loss is Okay

In [10]:
trainMSE_threshold=1e-2
assert sum(res["Final Train MSE"] >= trainMSE_threshold + res["sigma"]) == 0

In [ ]:
res[res["Final Train MSE"] >= trainMSE_threshold + res["sigma"]]["Final Train MSE"]

In [12]:
res = res[res["Final Train MSE"] < trainMSE_threshold + res["sigma"]] #filtering out bad fits

# Tuning Hyperparameters

##  determine the lambda parameter that gets the best Validation MSE for each (r,n,L)

In [ ]:
validationmse_vs_lambda = res.pivot_table(values="Validation MSE",index = ("r","sigma","n","L","Activations"),columns=["lambda"])
validationmse_vs_lambda

In [ ]:
bestlambda = validationmse_vs_lambda.idxmin(axis=1)
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    display(bestlambda)

In [ ]:
mask = [row["lambda"] == bestlambda[row["r"]][row["sigma"]][row["n"]][row["L"]][row["Activations"]] for rowindex,row in res.iterrows()]
res = res[mask]
res

##  determine the $L\ge3$ parameter that gets the best validation MSE for each (r,n)

In [ ]:
validationmse_vs_L = res.pivot_table(values="Validation MSE",index = ("r","sigma","n","Activations"),columns=["L"])
validationmse_vs_L = validationmse_vs_L.iloc[:,1:]
validationmse_vs_L

In [ ]:
bestL = validationmse_vs_L.idxmin(axis=1)
bestL

In [18]:
mask = [row["L"] == bestL[row["r"]][row["sigma"]][row["n"]][row["Activations"]] for rowindex,row in res.iterrows()]
bestLres = res[mask]

In [ ]:
bestLres.sort_values(by=['r','n',"sigma","Activations"])

## What are the chosen lambda and L for each model?

In [20]:
bestres = pd.concat((res[res["L"] == 2],bestLres))

In [ ]:
print(bestres.pivot_table(index=["r","sigma","n","Activations","L"],values=["lambda"]).shape)
bestres.pivot_table(index=["r","sigma","n","Activations","L"],values=["lambda"])

# Plotting

In [22]:
fontname = "Times New Roman"
import matplotlib
matplotlib.rcParams['mathtext.fontset'] = 'stix'
matplotlib.rcParams['font.family'] = 'STIXGeneral'

## Plots of L vs Validation error

In [ ]:
for std in res.sigma.unique():
    f, ax = plt.subplots(ncols=len(res.Activations.unique()),nrows=1, sharex=True, sharey=True, figsize=(10,4.8))
    for col,activation in enumerate(res.Activations.unique()):
        for n in res.n.unique():
                res_rnstd = res[(res.n == n) * (res["sigma"] == std) * (res.Activations == activation)]
                ax[col].scatter(res_rnstd.L,res_rnstd[["Validation MSE"]])
                ax[col].semilogy(res_rnstd.L,res_rnstd[["Validation MSE"]],label=rf"$n={n}$")
                for _,model in res_rnstd.iterrows():
                    text = rf'$\lambda = {model["lambda"]:.0e}$' + f'\nfit {model["Final Train MSE"]:.1e}\nwd{model["Final Weight Decay"]:.1e}'
        ax[col].set_xlabel("$L$ number of layers")
        ax[col].set_title(activation)
        if std > 0:
            ax[col].axhline(y=std**2, color='k', linestyle=':',label="$\sigma^2$")
        ax[0].set_ylabel("Validation MSE")
        f.suptitle(rf"Validation MSE for best $\lambda$ values, $\sigma$ = {std}")
    ax[0].legend()
    f.tight_layout()
    if int(std) == std:
        std = int(std)
    f.savefig("architecture_comparison"+f"_labelnoise{std}_ValidationMSE.pdf",dpi=300)
    plt.show()

## Performance metrics with/without linear layers

In [ ]:
res.r.unique()

In [ ]:
res.Activations.unique()

In [26]:
columnwidth = 6.17406722223
markersize = 4
colors = {  
    0   :"C0",
    0.25:"C1",
}
linestyles = {
    "Shallow":"-",
    "Linear Layers then ReLU":"--",
    'Deep ReLU':"-.",
    # 'ReLU then Linear Layers then ReLU':(0, (3, 1, 1, 1, 1, 1))
}
markers = {
    "Shallow":".",
    "Linear Layers then ReLU":"x",
    'Deep ReLU':"^",
    # 'ReLU then Linear Layers then ReLU':"v"
}

### generalization

In [ ]:
ax.shape

In [ ]:
#generalization and OOD errors
handles = [
    Line2D([0], [0], color=color, ls='-', label=rf"$\sigma =${sigma}") for sigma,color in colors.items()
]
handles += [
    Line2D([0], [0], color='k', ls=linestyles[activation], label=activation, marker = markers[activation], markersize=markersize) for activation in res.Activations.unique()
] 
handles += [
        Line2D([0], [0], color='k', ls=':', label='$\sigma^2$, irreducible error'),
]

f, ax = plt.subplots(ncols=2,nrows=2, sharex=True, sharey="row", figsize=(columnwidth,4.25))
for col,metric in enumerate(['In-Distribution','Out-of-Distribution']):
    standard_errors = metric[:-3] + 'SEM'
    #just the data without label noise in the first row
    for row,sigma in enumerate([0,0.25]):
        for activation in res.Activations.unique():
            curr = bestres[(bestres.Activations == activation) * (bestres.sigma == sigma)]
            print(curr.shape)
            points = curr[[metric + " MSE"]].values[:,0]
            ax[row,col].plot(curr.n,points,
                                linestyle=linestyles[activation],
                                marker=markers[activation],
                                markersize=markersize,
                                color=colors[sigma],
                                alpha=0.8)
            #horizontal dashed line for minimal possible MSE (ie sigma^2) in plots with label noise
            ax[-1,col].axhline(y=sigma**2, color=colors[sigma], linestyle=':',alpha=0.3)
        #plot set up
        ax[row,col].set_xscale("log",base=2)
        ax[row,col].set_xticks([2**k for k in range(6,12)])
        ax[row,col].set_yscale("log",base=10)
        ax[row,0].set_ylabel(f"MSE",wrap=True)
        ax[-1,col].set_xlabel("Number of training samples ($n$)")
        ax[row,col].minorticks_off()
        ax[0,col].set_title(metric)
f.legend(handles=handles, ncol=2, loc = 'upper center', bbox_to_anchor=(0.5,0.03))
plt.suptitle(f"Generalization across Architectures")
plt.tight_layout(pad=0.5,h_pad=1.08, w_pad=1.08)
plt.savefig("architecture_comparison"+f"Generalization.pdf",dpi=300,bbox_inches='tight')
plt.show()

### singular values

In [ ]:
handles = [
    Line2D([0], [0], color=color, ls='-', label=rf"$\sigma =${sigma}") for sigma,color in colors.items()
]
handles += [
    Line2D([0], [0], color='k', ls=linestyles[activation], label=activation, marker = markers[activation], markersize=markersize) for activation in res.Activations.unique()
] 
handles += [Line2D([0], [0], color='k', ls=':', label=r"effective rank tolerance, $\varepsilon = 10^{-3}$")]

ranktol = 1e-3
f, ax = plt.subplots(nrows=2, ncols=3, sharex=True, sharey=True, figsize=(columnwidth,4.25))#,5.75))
r = 1
for activation in res.Activations.unique():
    for nnum,n in enumerate(res.n.unique()):
        row = nnum // 3
        col = nnum % 3
        for sigma in res.sigma.unique():
            for activation in res.Activations.unique():
                print(activation,sigma,n)
                curr = bestres[(bestres.Activations == activation) * (bestres.sigma == sigma) * (bestres.n == n)]
                ax[row,col].semilogy(curr["Gradient Singular Values"].values[0]/np.sqrt(2048),
                    linestyle=linestyles[activation],
                    linewidth=1,
                    alpha=0.3,
                    marker=markers[activation],
                    markersize=markersize,
                    color=colors[sigma])
        ax[row,col].axhline(y=ranktol, color='k', linestyle=':',alpha=1, label = r"effective rank tolerance, $\varepsilon = 10^{-3}$")
        ax[row,col].set_xticks(list(range(4,20,5)),list(range(5,21,5)))
        ax[0,0].set_yticks([10**p for p in range(-9,3,2)])
        ax[row,col].set_title(rf"$r={r},n={n}$")
        ax[-1,col].set_xlabel(rf"Index, $k$")
        ax[row,col].set_ylim(10**(-9),10**(2.5))
f.legend(handles=handles, ncol=2, loc = 'upper center', bbox_to_anchor=(0.5,0.01))
plt.suptitle(r"Singular Values of Trained Networks, $\sigma_k(\hat{f})$, across Architectures")
plt.tight_layout(pad=0.5,h_pad=1.08, w_pad=1.08)
plt.savefig("architecture_comparison"+f"_labelnoise_sv.pdf",dpi=300, bbox_inches='tight')
plt.show()
handles.pop()

### active subspaces

In [ ]:
handles = [
    Line2D([0], [0], color=color, ls='-', label=rf"$\sigma =${sigma}") for sigma,color in colors.items()
]
handles += [
    Line2D([0], [0], color='k', ls=linestyles[activation], label=activation, marker = markers[activation], markersize=markersize) for activation in res.Activations.unique()
] 

#active subspace error plot
f, ax = plt.subplots(ncols=2,nrows=1, sharex=True, sharey=False, figsize=(columnwidth,2.75))
for col,metric in enumerate([r"Effective Index Rank, $\varepsilon = 10^{-3}$","Principal Angle (Degrees)"]):
    for sigma in res.sigma.unique():
        print(sigma)
        for activation in res.Activations.unique():
            curr = bestres[(bestres.Activations == activation) * (bestres.sigma == sigma)]
            if metric == "Principal Angle (Degrees)":
                points = curr[[metric]].values[:,0]
            elif metric == r"Effective Index Rank, $\varepsilon = 10^{-3}$":
                points = (np.array(curr["Gradient Singular Values"].tolist())/np.sqrt(2048) > ranktol).sum(axis=1)
                ax[col].set_yticks(np.arange(0,21,5))
                ax[col].set_ylim(0,20.5)
            ax[col].plot(curr.n,points,
                                linestyle=linestyles[activation],
                                color=colors[sigma],
                                marker=markers[activation],
                                markersize=markersize,
                                alpha=0.8)
    #plot set up
    ax[col].set_title(metric[:15] + '\n' + metric[16:])
    ax[0].set_yticks(range(21), minor=True)
    ax[col].set_xscale("log",base=2)
    ax[col].set_xticks([2**k for k in range(6,12)])
    ax[1].set_xlabel("Number of training samples ($n$)")
    ax[col].minorticks_on()
f.legend(handles=handles, ncol=2, loc = 'upper center', bbox_to_anchor=(0.5,0.03))
plt.suptitle(f"Active Subspaces across Architectures")
plt.tight_layout(pad=0.5,h_pad=0.5, w_pad=0.5)
plt.savefig("architecture_comparison"+f"Active Subspaces.pdf",dpi=300,bbox_inches='tight')
plt.show()

## training

In [ ]:
handles = [
    Line2D([0], [0], color=color, ls='-', label=rf"$\sigma =${sigma}") for sigma,color in colors.items()
]
handles += [
    Line2D([0], [0], color='k', ls=linestyles[activation], label=activation, marker = markers[activation], markersize=markersize) for activation in res.Activations.unique()
] 

#training time plot
for metric in ["Weight Decay","Train MSE"]:
    f, ax = plt.subplots(ncols=3,nrows=len(res.n.unique())//3, sharex=True, sharey=True, figsize=(columnwidth,4))
    for nnum,n in enumerate(res.n.unique()):
        row = nnum//3
        col = nnum % 3
        for sigma in res.sigma.unique():
            for activation in res.Activations.unique():
                curr = bestres[(bestres.Activations == activation) * (bestres.sigma == sigma) * (bestres.n == n)]
                assert curr[metric].shape[0] == 1 # make sure there's only one row
                values_to_plot = curr[metric].iloc[0]
                epochs = len(values_to_plot)
                ax[row,col].plot(np.arange(epochs),values_to_plot,
                                    linestyle=linestyles[activation],
                                    color=colors[sigma],
                                    linewidth = 1,
                                    alpha=0.8)
                ax[row,col].scatter(np.arange(epochs)[::15_000],values_to_plot[::15_000],
                                    linestyle=linestyles[activation],
                                    color=colors[sigma],
                                    marker=markers[activation],
                                    linewidth = 1,
                                    alpha=0.8)
            #plot set up
            ax[row,col].set_title(f"n = {n}")
            ax[row,col].set_yscale("log",base=10)
            ax[row,col].minorticks_on()
            ax[-1,col].set_xlabel("Epochs")
            ticks = np.arange(15_000,epochs,step=15_000)
            ax[-1,col].set_xticks(ticks=ticks, labels=[str(t)[:-3]+'k' for t in ticks])
    f.legend(handles=handles, ncol=2, loc = 'upper center', bbox_to_anchor=(0.5,0.01))
    plt.suptitle(metric+" During Training across Architectures")
    plt.tight_layout(pad=0.5,h_pad=0.5, w_pad=0.5)
    plt.savefig("architecture_comparison"+metric+"training.pdf",dpi=300,bbox_inches='tight')
    plt.show()